#Ml flow

In [1]:
import numpy

In [2]:
# ===================== INSTALL =====================
!pip install optuna mlflow torch torchvision

# ===================== IMPORTS =====================
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import mlflow
import mlflow.pytorch

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

# ===================== DEVICE =====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ===================== DATA =====================
df = pd.read_csv("fmnist_small.csv")

X = df.iloc[:, 1:].values / 255.0
y = df.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_data = CustomDataset(X_train, y_train)
test_data  = CustomDataset(X_test, y_test)

# ===================== MODEL =====================
class MyNN(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, neurons, dropout):
        super().__init__()
        layers = []

        for _ in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neurons))
            layers.append(nn.BatchNorm1d(neurons))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = neurons

        layers.append(nn.Linear(neurons, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ===================== MLFLOW =====================
mlflow.set_experiment("FMNIST_Optuna_DeepLearning")

# ===================== OPTUNA OBJECTIVE =====================
def objective(trial):

    params = {
        "num_hidden_layers": trial.suggest_int("num_hidden_layers", 1, 5),
        "neurons_per_layer": trial.suggest_int("neurons_per_layer", 32, 256, step=32),
        "epochs": trial.suggest_int("epochs", 10, 50, step=10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "dropout_rate": trial.suggest_float("dropout_rate", 0.1, 0.5),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    }

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)

        train_loader = DataLoader(
            train_data, batch_size=params["batch_size"], shuffle=True
        )
        test_loader = DataLoader(
            test_data, batch_size=params["batch_size"], shuffle=False
        )

        model = MyNN(
            784, 10,
            params["num_hidden_layers"],
            params["neurons_per_layer"],
            params["dropout_rate"]
        ).to(device)

        criterion = nn.CrossEntropyLoss()

        if params["optimizer"] == "Adam":
            optimizer = optim.Adam(model.parameters(),
                                   lr=params["learning_rate"],
                                   weight_decay=params["weight_decay"])
        elif params["optimizer"] == "SGD":
            optimizer = optim.SGD(model.parameters(),
                                  lr=params["learning_rate"],
                                  weight_decay=params["weight_decay"])
        else:
            optimizer = optim.RMSprop(model.parameters(),
                                      lr=params["learning_rate"],
                                      weight_decay=params["weight_decay"])

        # ===== TRAIN =====
        model.train()
        for _ in range(params["epochs"]):
            for x_batch, y_batch in train_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                loss = criterion(model(x_batch), y_batch)
                loss.backward()
                optimizer.step()

        # ===== EVAL =====
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x_batch, y_batch in test_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                preds = torch.argmax(model(x_batch), dim=1)
                total += y_batch.size(0)
                correct += (preds == y_batch).sum().item()

        accuracy = correct / total
        mlflow.log_metric("accuracy", accuracy)
        mlflow.pytorch.log_model(model, "model")

        return accuracy

# ===================== RUN OPTUNA =====================
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best Accuracy:", study.best_value)
print("Best Params:", study.best_params)


c:\Users\Lenovo\anaconda3\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Using device: cpu


2025/12/27 14:01:38 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/27 14:01:39 INFO mlflow.store.db.utils: Updating database tables
2025/12/27 14:01:39 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/27 14:01:39 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/27 14:01:40 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2025/12/27 14:01:40 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2025/12/27 14:01:40 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2025/12/27 14:01:41 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2025/12/27 14:01:41 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2025/12/27 14:01:41 INFO alembic.runtime.migration: Running 

Best Accuracy: 0.8391666666666666
Best Params: {'num_hidden_layers': 1, 'neurons_per_layer': 224, 'epochs': 30, 'learning_rate': 0.0021073436744176144, 'dropout_rate': 0.2783088940003031, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 0.00013199223968246766}


In [3]:
import shutil
import os

mlruns_dir = "/content/mlruns"
zip_path = "/content/mlruns.zip"

if os.path.exists(mlruns_dir):
    shutil.make_archive("/content/mlruns", "zip", mlruns_dir)
    print("mlruns zipped successfully!")
else:
    print("mlruns folder not found.")


mlruns folder not found.
